In [49]:
import pandas as pd
import numpy as np
import os

In [50]:
def pooling(x, power = 4):
    x_pooling = np.power(np.mean(np.power(x, power)), 1 / power)
    return x_pooling

In [51]:
dataset_scores = pd.read_pickle('dataset_scores.pkl')
fr_original = pd.read_pickle('fr_original.pkl')
nr_metrics = pd.read_pickle('nr_metrics.pkl')

In [52]:
nr_metrics = nr_metrics.drop(['Slice', 'Blackout', 'Freezing','Letterbox', 'Pillarbox', ],axis = 1)

In [53]:
fr_original.index.names = ['Video', 'Frame']

In [54]:
nr_metrics.describe()

,Blockiness,SA,Blockloss,Blur,TA,Exposure(bri),Contrast,Interlace,Noise,Flickering
count,65955.000000,65955.000000,65955.000000,65955.000000,65955.000000,65955.000000,65955.000000,65955.00000,65955.000000,65955.000000
mean,0.915915,48.944504,4.217661,5.411046,10.177859,125.505117,67.154017,0.00200,0.437919,-0.793173
std,0.063227,24.926557,5.940220,1.736381,7.647164,5.245157,32.273976,0.00227,0.375433,0.546725
min,0.414150,4.433490,0.000000,2.264000,0.000000,68.000000,2.655120,0.00000,0.000000,-1.000000
25%,0.891970,30.285290,0.804690,4.097000,3.589405,124.000000,44.782700,0.00076,0.256000,-1.000000
50%,0.931540,46.278310,2.097780,5.098000,8.516590,126.000000,55.850130,0.00140,0.314600,-1.000000
75%,0.958820,62.174365,5.109535,6.288500,15.305515,127.000000,86.565295,0.00237,0.430400,-1.000000
max,1.017000,133.286830,101.882810,13.769000,75.560340,145.000000,142.082680,0.02094,3.999700,0.931360


In [55]:
fr_original.describe()

,msssim,psnr,psnrhvs,psnrhvsm,ssim,vifp
count,63974.000000,64083.000000,64083.000000,64083.000000,64083.000000,64083.000000
mean,0.942037,36.313163,33.712847,37.595412,0.933568,0.715966
std,0.168620,7.506900,7.726080,9.317310,0.191778,0.185366
min,0.073115,8.010797,4.166093,4.314585,-0.064055,0.007650
25%,0.972138,33.939350,31.196902,33.897365,0.968377,0.666906
50%,0.986717,37.189877,34.473221,37.961124,0.984738,0.752600
75%,0.994600,41.058983,38.840562,43.965137,0.992702,0.834025
max,0.999984,67.235413,65.132843,70.459534,0.999954,0.966717


In [56]:
for i in pd.unique(dataset_scores.index):
    if 'ERP' not in i:
        dataset_scores = dataset_scores.drop(index=i)

In [57]:
labels =pd.DataFrame(columns=['Video','Raw', 'DMOS'])

In [58]:
for i in pd.unique(dataset_scores.index):
    element = np.mean(dataset_scores.loc[i])
    labels = labels.append({'Video': i, 'Raw': element.Raw, 'DMOS': element.DMOS}, ignore_index=True)

In [59]:
labels = labels.set_index(['Video'])
labels = labels.sort_index()

In [60]:
extraction_fr = ['max', 'min', 'mean', 'std']
feature_fr = pd.DataFrame()
for idx in pd.unique(fr_original.index.get_level_values(level = 0)):
    temp = fr_original.loc[idx]
    temp = temp.apply(pooling, axis = 0)
    temp.name = idx
    feature_fr = feature_fr.append(temp)

In [61]:
extraction_nr = ['max', 'mean', 'std']
feature_nr = pd.DataFrame()
for idx in pd.unique(nr_metrics.index.get_level_values(level = 0)):
    temp = nr_metrics.loc[idx]
    temp = temp.apply(pooling, axis = 0)
    temp.name = idx
    feature_nr = feature_nr.append(temp)

In [62]:
feature = pd.concat([feature_fr, feature_nr], axis = 1, join='inner')

In [63]:
training = pd.concat([feature, labels], axis = 1, join='inner')

In [64]:
training = training.sort_index()

In [65]:
labels = training.loc[:,'DMOS']
features = training.drop(columns=['Raw', 'DMOS'])

In [66]:
features.head()

,msssim,psnr,psnrhvs,psnrhvsm,ssim,vifp,Blockiness,Blockloss,Blur,Contrast,Exposure(bri),Flickering,Interlace,Noise,SA,TA
G10BoatInPark_ERP_4096x2048_fps30_qp27_14547k.mp4,0.993424,41.829451,39.401728,43.855814,0.989905,0.818723,0.918001,4.662472,6.318822,56.274408,122.973648,0.973981,0.004543,0.385048,48.017774,12.572215
G10BoatInPark_ERP_4096x2048_fps30_qp37_3270k.mp4,0.978172,37.045777,34.074029,36.540714,0.975696,0.721947,0.923023,5.707591,6.802874,56.220687,122.235861,0.974016,0.001439,0.250219,45.613703,12.228131
G10BoatInPark_ERP_4096x2048_fps30_qp42_1507k.mp4,0.959965,34.534499,31.430402,33.216521,0.962685,0.672978,0.923070,6.641128,7.194109,56.157094,121.932088,0.973082,0.001407,0.243489,42.835611,11.782598
G10BodybuildingWorkout_ERP_7680x3840_fps29.97_qp27_6105k.mp4,0.997337,46.713699,44.069635,48.525252,0.997130,0.915188,0.941557,1.083126,10.663758,107.170623,125.000000,0.975835,0.000408,0.691092,55.521111,3.173898
G10BodybuildingWorkout_ERP_7680x3840_fps29.97_qp37_913k.mp4,0.993475,42.391911,39.490941,42.097111,0.994158,0.873338,0.916841,0.990191,10.873596,107.176925,124.268885,0.970290,0.001945,0.350776,55.119636,3.099582


In [67]:
labels.head()

G10BoatInPark_ERP_4096x2048_fps30_qp27_14547k.mp4               34.140535
G10BoatInPark_ERP_4096x2048_fps30_qp37_3270k.mp4                41.399333
G10BoatInPark_ERP_4096x2048_fps30_qp42_1507k.mp4                54.386335
G10BodybuildingWorkout_ERP_7680x3840_fps29.97_qp27_6105k.mp4    35.192198
G10BodybuildingWorkout_ERP_7680x3840_fps29.97_qp37_913k.mp4     56.533862
Name: DMOS, dtype: float64

In [68]:
labels.to_pickle('labels.pkl')

In [69]:
features.to_pickle('features.pkl')

In [70]:
labels.size

177